<a href="https://colab.research.google.com/github/silphy1001/YouTubeDataAPIExercise/blob/master/YouTubeDataAPIExercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import urllib.request
import urllib.parse
import json
from collections import OrderedDict
import pprint
import sqlite3
import os
from datetime import datetime

from google.colab import drive

In [ ]:
# Google Drive上にデータベースを作成する
drive_path = "/content/drive/My Drive/ネタ/VTuber統計データ/"
db_name = "vtuber_stats.db"
db_file = drive_path + db_name
DB_EXISTS_FLAG = True

if not os.path.exists(drive_path):
  # Google Driveがマウントされていない時はマウントする
  drive.mount('/content/drive')
if not os.path.exists(db_file):
  DB_EXISTS_FLAG = False

# DBファイルが存在しない時は新規作成し、存在する時はそのまま接続する
con = sqlite3.connect(db_file)
cur = con.cursor()

# DBを新規作成した場合はテーブル定義を行う
if not DB_EXISTS_FLAG:
  # vtstatsというテーブルを作成
  cur.execute(
      'CREATE TABLE vtstats(id INTEGER PRIMARY KEY AUTOINCREMENT, \
       chtitle TEXT, chid TEXT, subscribercount INTEGER, viewcount INTEGER, \
       videocount INTEGER, registdate TEXT)' \
  )
  con.commit()

# テーブルを確認
cur.execute('SELECT * FROM vtstats')
pprint.pprint(cur.fetchall(), width=50)

cur.close()
con.close()

In [ ]:
# 例外処理は後で追加する。
# 例）
## HTTP通信エラー 4xx系、5xx系
## YouTube Data API v3 のAPIエラー
## ...etc

# 定数定義
API_KEY = input()
BASE_URL = 'https://www.googleapis.com/youtube/v3'
HEADERS = {
    'Content-Type': 'application/json',
}

In [ ]:
# データ定義
profiles = {
            '花京院ちえり': {
              'chtitle': '',
              'chid': 'UCP9ZgeIJ3Ri9En69R0kJc9Q',
              'subscribercount': 0,
              'viewcount': 0,
              'videocount': 0,
              'registdate': '2020-01-01'
            },
            '八重沢なとり': {
              'chtitle': '',
              'chid': 'UC1519-d1jzGiL1MPTxEdtSA',
              'subscribercount': 0,
              'viewcount': 0,
              'videocount': 0,
              'registdate': '2020-01-01'
            },
            '神楽すず': {
              'chtitle': '',
              'chid': 'UCUZ5AlC3rTlM-rA2cj5RP6w',
              'subscribercount': 0,
              'viewcount': 0,
              'videocount': 0,
              'registdate': '2020-01-01'
            },
            'ヤマト イオリ': {
              'chtitle': '',
              'chid': 'UCyb-cllCkMREr9de-hoiDrg',
              'subscribercount': 0,
              'viewcount': 0,
              'videocount': 0,
              'registdate': '2020-01-01'
            },
           '金剛いろは': {
              'chtitle': '',
              'chid': 'UCiGcHHHT3kBB1IGOrv7f3qQ',
              'subscribercount': 0,
              'viewcount': 0,
              'videocount': 0,
              'registdate': '2020-01-01'
            },
           'もこ田めめめ': {
              'chtitle': '',
              'chid': 'UCz6Gi81kE6p5cdW1rT0ixqw',
              'subscribercount': 0,
              'viewcount': 0,
              'videocount': 0,
              'registdate': '2020-01-01'
            },
           'カルロ・ピノ': {
              'chtitle': '',
              'chid': 'UCMzxQ58QL4NNbWghGymtHvw',
              'subscribercount': 0,
              'viewcount': 0,
              'videocount': 0,
              'registdate': '2020-01-01'
            },
           '月ノ美兎': {
              'chtitle': '',
              'chid': 'UCD-miitqNY3nyukJ4Fnf4_A',
              'subscribercount': 0,
              'viewcount': 0,
              'videocount': 0,
              'registdate': '2020-01-01'
            },
           '名取さな': {
              'chtitle': '',
              'chid': 'UCIdEIHpS0TdkqRkHL5OkLtA',
              'subscribercount': 0,
              'viewcount': 0,
              'videocount': 0,
              'registdate': '2020-01-01'
            },
}

In [ ]:
# 参照 - YouTube Data API > Guides and Tutorials > チャンネルIDへの対応
## https://developers.google.com/youtube/v3/guides/working_with_channel_ids?hl=ja

# チャンネルIDを使ってチャンネル名を取り出す
for name, details in profiles.items():
  command = '/channels' \
          + '?key=' + API_KEY \
          + '&part=snippet' \
          + '&id=' + details['chid']
  published_url = BASE_URL + command

  req = urllib.request.Request(published_url, headers=HEADERS)
  with urllib.request.urlopen(req) as res:
    body = json.loads(res.read().decode('utf-8'))
    chtitle = body['items'][0]['snippet']['localized']['title'] # itemsがリストになっているので0番目を取り出す
    print("チャンネル名: " + chtitle)
    profiles[name]['chtitle'] = chtitle
  print(published_url)

In [ ]:
# チャンネル情報を取得する
for k in profiles.keys():
  channel_id = profiles[k]['chid']
  command = '/channels?key=' + API_KEY \
          + '&part=statistics' \
          + '&id=' + channel_id
  published_url = BASE_URL + command
  print(published_url)

  req = urllib.request.Request(published_url, headers=HEADERS)
  with urllib.request.urlopen(req) as res:
    body = json.loads(res.read().decode('utf-8'))
    profiles[k]['subscribercount'] = int(body['items'][0]['statistics']['subscriberCount'])
    profiles[k]['viewcount'] = int(body['items'][0]['statistics']['viewCount'])
    profiles[k]['videocount'] = int(body['items'][0]['statistics']['videoCount'])
    profiles[k]['registdate'] = datetime.now().isoformat()